In [2]:
#  kraken2 report to OTU table and Tax table script
import pandas as pd
import numpy as np
import os
from pandas import read_csv, DataFrame
from ete3 import NCBITaxa
import click
#  one-time download, later just updates silently
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()

In [ ]:
#  tryin' click for the 1st time
@click.command()
@click.argument('folder_path')
@click.argument('outdir')
@click.option('--path', default='./', help='Path to Kraken2 reports folder.')
@click.option('--outdir',default ='./', help='Folder to store resulting OTU table and Tax table.')
def read_path_to_data(folder_path):
    """Read path to Kraken2 reports folder"""
        click.echo(f'Received input argument: {folder_path}')

if __name__ == '__main__':
    read_path_to_data()

In [3]:
folder_path = '/home/ignatsonets/kraken2otu/reports'  # Replace with the actual path to your folder, hardcoded now
file_extension = '.report'

# create an empty DataFrame to store the data
merged_df = pd.DataFrame()
merged_df_taxid = pd.DataFrame()
# iterate over the files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(file_extension):
        file_path = os.path.join(folder_path, file_name)
        
        #  extract the desired columns from each file. For Tax table it is 5th col, for OTU table it is 3rd and 6th columns
        temp_df = pd.read_csv(file_path, delimiter='\t', skiprows=1, usecols=[2, 5], header=None)
        temp_df_taxid = pd.read_csv(file_path, delimiter='\t', skiprows=1, usecols=[4], header=None)
        
        #  extract the sample number from the file name
        sample_number = os.path.splitext(file_name)[0]
        
        #  add the sample number as a new column in the DataFrame
        temp_df['sample_number'] = sample_number
        temp_df_taxid['sample_number'] = sample_number
        
        # concatenate the data into one DataFrame
        merged_df = pd.concat([merged_df, temp_df])
        merged_df_taxid = pd.concat([merged_df_taxid, temp_df_taxid])
#  renaming columns for the pre-OTU table (here in long format)
merged_df.columns=["Count", "Taxon",  "Sample"]
merged_df= merged_df.reset_index(drop=True)
#  Removing leading spaces of taxonomic labels with regex
merged_df = merged_df.replace(r"^ +| +$", r"", regex=True)
#  The same renaming for Tax table
merged_df_taxid.columns =['TaxID', 'Sample']
merged_df_taxid = merged_df_taxid.reset_index(drop=True)

In [4]:
#  creating an OTU table
otu_table = {}

for sample in merged_df['Sample'].unique():
    
    sample_subset = merged_df[merged_df['Sample'] == sample]
    otu_table[sample] = {sample_subset['Taxon'][idx] : sample_subset['Count'][idx] for idx in sample_subset.index}
otu_table = DataFrame(otu_table).fillna(0).T
zero_cols = otu_table.columns[(otu_table <=0).all()]
otu_table.drop(labels=zero_cols, axis=1, inplace=True)
otu_table.head(10)

,root,cellular organisms,Eukaryota,Sar,Apicomplexa,Aconoidasida,Plasmodium,Plasmodium relictum,Plasmodium (Vinckeia),Plasmodium vinckei vinckei,...,Methanothermobacter thermautotrophicus str. Delta H,Methanococcus maripaludis S2,Pepper mild mottle virus,Flavobacterium phage vB_FspM_immuto_2-6A,Pahexavirus PHL308M00,Propionibacterium phage PHL085N00,Klebsiella virus KpV2811,Streptococcus phage phiARI0468-4,Mycobacterium phage BabyRay,Lactococcus phage 63302
23NVShed6_o,15118.0,2665209.0,34758.0,41584.0,3447.0,2085.0,10036.0,600444.0,125.0,6238.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed7_o,5035.0,8991124.0,129212.0,5891.0,6963.0,2101.0,4077.0,262389.0,180.0,5296.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed7_a,2469.0,637427.0,113551.0,14671.0,21400.0,2637.0,21980.0,23438.0,390.0,828.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed5_o,8755.0,14763576.0,79820.0,8208.0,7559.0,4723.0,5760.0,171954.0,243.0,7547.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed3_a,3023.0,4767232.0,120723.0,13159.0,21831.0,6494.0,9383.0,39426.0,293.0,1569.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed2_o,8194.0,25622271.0,199961.0,11062.0,16252.0,2303.0,9263.0,19950.0,569.0,3836.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed4_a,7130.0,1880711.0,134341.0,16104.0,30919.0,11590.0,25477.0,27119.0,1439.0,1542.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed4_o,3926.0,6362824.0,88264.0,14715.0,15261.0,6129.0,8433.0,244611.0,206.0,7055.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed3_o,6571.0,11950038.0,243590.0,20889.0,21375.0,1115.0,9612.0,133699.0,338.0,4485.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23NVShed1_o,2668.0,486742.0,59305.0,1516.0,17610.0,12463.0,12707.0,168175.0,97.0,441.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
#  smol df just for testing
taxid_test = merged_df_taxid.sample(n=3)
taxid_test

,TaxID,Sample
75984,79604,23NVShed4_a
111240,110505,23NVShed1_o
114016,1301283,23NVShed1_o


In [67]:
taxo_names_df = pd.DataFrame()
for _, row in taxid_test.iterrows():
    taxid = int(row['TaxID'])
    lineage = ncbi.get_lineage(taxid)
    #print(lineage)
    names = ncbi.get_taxid_translator(lineage)
    print([names[taxid] for taxid in lineage])
#  later add filtration to save only necessary ranks
#  this might help i guess: 
#  https://stackoverflow.com/questions/36503042/how-to-get-taxonomic-specific-ids-for-kingdom-phylum-class-order-family-gen/36517712#36517712?newreg=1131bb17c142425887049bc55eff0c9f


['root', 'cellular organisms', 'Bacteria', 'Terrabacteria group', 'Actinomycetota', 'Coriobacteriia', 'Eggerthellales', 'Eggerthellaceae', 'Denitrobacterium', 'Denitrobacterium detoxificans']
['root', 'cellular organisms', 'Bacteria', 'Terrabacteria group', 'Actinomycetota', 'Actinomycetes', 'Mycobacteriales', 'Mycobacteriaceae', 'Mycobacterium', 'Mycobacterium heckeshornense']
['root', 'cellular organisms', 'Bacteria', 'Terrabacteria group', 'Cyanobacteriota/Melainabacteria group', 'Cyanobacteriota', 'Cyanophyceae', 'Oscillatoriophycideae']
